In [71]:
import os
import sys
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import set_config

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering

df = pd.read_csv(folder_path + '/SBA_Cleaned_maybe.csv')

In [72]:
y = df['Approve']
X = df.drop('Approve', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, test_size=0.05, random_state=42, stratify=y)

In [73]:
numerical_columns_selector = X.select_dtypes(exclude='object').columns
categorical_columns_selector = X.select_dtypes(include='object').columns

numerical_columns = X[numerical_columns_selector]
categorical_columns = X[categorical_columns_selector]

In [74]:
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  
    ('scaler', StandardScaler())  
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),  
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  
])

In [75]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_columns_selector),
        ('cat', categorical_transformer, categorical_columns_selector)
    ],
    remainder='passthrough'  
)

In [76]:
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('gbm', GradientBoostingClassifier())  
])

In [77]:
param_grid = {
    'gbm__n_estimators': [100],  
    'gbm__learning_rate': [0.05, 0.1, 0.2],  
    'gbm__max_depth': [5],
    'gbm__subsample': [0.7, 1.0],
    'gbm__max_features': ['auto', 'sqrt', 'log2'],
    'gbm__loss': ['deviance', 'exponential'],
    'gbm__criterion': ['friedman_mse', 'mse', 'mae'],
    'gbm__init': ['zero', 'random'],
    'gbm__random_state': [42]
}

In [79]:
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='f1_macro')
grid_search.fit(X_train, y_train)

ValueError: Invalid parameter 'n_jobs' for estimator GradientBoostingClassifier(init='zero', learning_rate=0.05, loss='deviance',
                           max_depth=5, max_features='auto'). Valid parameters are: ['ccp_alpha', 'criterion', 'init', 'learning_rate', 'loss', 'max_depth', 'max_features', 'max_leaf_nodes', 'min_impurity_decrease', 'min_samples_leaf', 'min_samples_split', 'min_weight_fraction_leaf', 'n_estimators', 'n_iter_no_change', 'random_state', 'subsample', 'tol', 'validation_fraction', 'verbose', 'warm_start'].

In [ ]:
print("Meilleurs hyperparamètres trouvés :")
print(grid_search.best_params_)

In [ ]:
print("Meilleure précision trouvée :", grid_search.best_score_)

In [ ]:
best_model = grid_search.best_estimator_
predictions = best_model.predict(X_test)

In [ ]:
f1_macro = f1_score(y_test, predictions, average='macro')
print("F1-score macro sur l'ensemble de test :", f1_macro)

In [ ]:
# param_grid = {
#     'n_estimators': [50, 100, 200],
#     'learning_rate': [0.05, 0.1, 0.2],
#     'max_depth': [3, 4, 5],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'subsample': [0.8, 0.9, 1.0],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'loss': ['deviance', 'exponential'],
#     'criterion': ['friedman_mse', 'mse', 'mae'],
#     'min_impurity_decrease': [0.0, 0.1, 0.2],
#     'min_impurity_split': [None, 0.1, 0.2],
#     'init': ['zero', 'random'],
#     'random_state': [42], 
#     'n_jobs': [-1]  
# }